In [ ]:
import datetime
from glob import glob
import os
import re

import pandas as pd
import qgrid


# Excel to Pandas (and Back)

### Download the data set

For this demo we're using the [Online Retail Data Set](http://archive.ics.uci.edu/ml/datasets/Online+Retail/) which can be found on the UCI Machine Learning Repository. 

Run the cells below to download the data programmatically. 

**Note:** If you navigate to the page linked above, you may find that the "Data Folder" link is broken. It should link to the following:  https://archive.ics.uci.edu/ml/machine-learning-databases/00352. 

In [ ]:
if not os.path.exists('data'):
    os.mkdir('data')
    
if not os.path.exists('data/monthly'):
    os.mkdir('data/monthly')

In [ ]:
!wget -nc -O data/online-retail.xlsx \
https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx

### Prepare the data

The data set describes online transactions taking place from December 2010 to December 2011. We split them into monthly spreadsheets for the sake of demonstration, since we'll be covering how to combine data from multiple files and how to automate tasks across multiple files.

In [ ]:
data_path = 'data/online-retail.xlsx'

In [ ]:
def convert_to_monthly(data_path):
    
    printout = 'Reading data from {}:'.format(data_path)
    print(printout)
    print('-'*len(printout))
    
    df = pd.read_excel(data_path)
    df['MonthYear'] = df['InvoiceDate'].dt.strftime('%b-%Y')

    for month_year in df['MonthYear'].unique():
        monthly_df = df[df['MonthYear'] == month_year].drop('MonthYear', 1)
        file_path = os.path.join('data/monthly', 'transactions-' + month_year + '.xlsx')
        print(' Writing file: {}'.format(file_path))
        monthly_df.to_excel(file_path, index=True)
        
    print('-'*len(printout))
    print('Complete.')
    

In [ ]:
convert_to_monthly(data_path)

## Reading from an Excel file

### Specify the path to the file

In [ ]:
sheet_path = 'data/monthly/transactions-Dec-2010.xlsx'

In [ ]:
def load_df(sheet_path):
    df = pd.read_excel(sheet_path, index_col=0)
    df['InvoiceNo'] = df['InvoiceNo'].astype(str)
    df['StockCode'] = df['StockCode'].astype(str)
    return df

In [ ]:
df = load_df(sheet_path)
df.head()

### Investigating missing values

In [ ]:
df.isna().mean()

In [ ]:
df[df['Description'].isna()].head(10)

**It looks like items without a description have a unit price of 0.0. Let's confirm this and omit these, for now.**

In [ ]:
print((df[df['Description'].isna()]['UnitPrice'] == 0).all())

def remove_items_without_description(df):
    return df[~df['Description'].isna()]

df = remove_items_without_description(df)

## Analysis

Let's start looking into something simple: what products are selling well? We'll need to do some *aggregation* over the product identifiers - the StockCodes. First, let's build a data frame of the products we find here. 

In [ ]:
products = df.groupby('StockCode')[['Description', 'UnitPrice']].first()
products.head()

Note that the last two rows describe different colors / variants of the same product. Should these be counted as separate products? 

In [ ]:
def separate_code_and_color(df):
    
    df[['StockCodeBase', 'Color']] = df['StockCode'].str.extract(r'(\d+)([A-Z])*')
    df['Color'] = df['Color'].fillna(-1)
    
    return df

In [ ]:
df = separate_code_and_color(df)

df.head()

In [ ]:
def get_product_df(df):
    
    products = (df.groupby('StockCodeBase')[['Description','Color', 'UnitPrice']]
     .agg(
         Description=('Description', 'first'),
         n_colors=('Color', 'nunique'),
         UnitPrice=('UnitPrice', 'first')
     ))
    
    return products

get_product_df(df).head()

## Top-selling items

In [ ]:
def top_selling_items(df, n_items=10):
    df['GrossRevenue'] = df['UnitPrice'] * df['Quantity']
    
    top_items = (df.groupby('StockCodeBase')[['Description', 'UnitPrice', 'Quantity', 'GrossRevenue']]
                 .agg(
                     Description=('Description', 'first'),
                     UnitPrice=('UnitPrice', 'first'),
                     Quantity=('Quantity', 'sum'),
                     TotalGrossRevenue=('GrossRevenue', 'sum')
                 )
                ).sort_values(by='TotalGrossRevenue', ascending=False)
    
    return top_items.iloc[:n_items, :]

top_selling_items(df)

## Working with strings

In [ ]:
df[df['Description'].str.contains('UNION')].sample(5)

In [ ]:
df[df['Description'].str.contains('UNION')]['Country'].value_counts()

### Item colors with regex

Earlier, we saw that the StockCode contained some information about the colors/variants of items, but it didn't help us compare across multiple items. 

Now, we'll use *regular expressions* (regex) to attempt to locate and extract a standard color from the item description. 

In [ ]:
color_reg = re.compile(r'((?:BLACK)|(?:WHITE)|(?:RED)|(?:GREEN)|(?:BLUE)|(?:PINK))')


In [ ]:
df['DescriptionColor'] = (df['Description']
                          .str.extract(color_reg)
                          .fillna('None'))
df.head()

In [ ]:
def top_selling_colors(df):
    color_reg = re.compile(r'((?:BLACK)|(?:WHITE)|(?:RED)|(?:GREEN)|(?:BLUE)|(?:PINK))')
    df['DescriptionColor'] = df['Description'].str.extract(color_reg).fillna('None')
    
    top_colors = (df
                  .groupby('DescriptionColor')['GrossRevenue']
                  .sum()
                  .sort_values(ascending=False)
                  .reset_index())
    return top_colors

In [ ]:
top_selling_colors(df)

## Working with Dates

The data description page mentions that many of the customers are wholesalers. We might expect that business customers exhibit different shopping patterns from non-business customers. For instance, we might wonder if business customers make more purchases on week days and during business hours.

Whether or not this is the case, Pandas makes it easy to extract this temporal information and use it to filter and process data.

We can use the `.dt` accessor to utilize datetime methods like `.year`, `.month`', `.hour`, and even `.dayofweek`.

In [ ]:
df['InvoiceDate'].dt.dayofweek.value_counts()

Interestingly enough, it appears there are no Saturday transactions in the data set. Let's see what we can do with time of day.

In [ ]:
df['InvoiceDate'].dt.hour.plot.hist()

These methods can be used to create filters for the data as well. 

Here we make a `business_hours` filter that keeps only transactions occurring between 9AM-5PM Monday through Friday.

In [ ]:
filter_9_to_5 = (df['InvoiceDate'].dt.hour >=9) & (df['InvoiceDate'].dt.hour <17)
filter_weekday = df['InvoiceDate'].dt.dayofweek <=5

In [ ]:
df_weekday = df[filter_9_to_5 & filter_weekday]
df_weekday

## Writing to an Excel file

For simple Excel output, we can use the Pandas `.to_excel` method. For more advanced usage, we use the [XlsxWriter](https://xlsxwriter.readthedocs.io/) library. With the latter, we can create more sophisticated files with multiple sheets and even charts.

In [ ]:
if not os.path.exists('output'):
    os.mkdir('output')


In [ ]:
file_path = os.path.join('output', 'Dec-2010'+ '.xlsx')

with pd.ExcelWriter(file_path) as writer:
    df.to_excel(writer, sheet_name='original', index=False)
    get_product_df(df).reset_index().to_excel(writer, sheet_name='products', index=False)
    top_selling_items(df).reset_index().to_excel(writer,sheet_name='top-items', index=False)
    top_selling_colors(df).to_excel(writer,sheet_name='top-colors', index=False)

## Automation

Now, let's do this to all the monthly files we have!

In [ ]:
def load_and_process_df(path):
    print("Loading {}".format(path))
    df = load_df(path)
    df = remove_items_without_description(df)
    df = separate_code_and_color(df)
    
    return df

In [ ]:
def generate_report(df, name):
    
    file_path = os.path.join('output', '{}.xlsx'.format(name))

    print('Writing {}'.format(file_path))
    with pd.ExcelWriter(file_path) as writer:
        df.to_excel(writer, sheet_name='original', index=False)
        get_product_df(df).reset_index().to_excel(writer, sheet_name='products', index=False)
        top_selling_items(df).reset_index().to_excel(writer,sheet_name='top-items', index=False)
        top_selling_colors(df).to_excel(writer,sheet_name='top-colors', index=False)

In [ ]:
monthly_report_paths = glob('data/monthly/*')
monthly_report_paths

In [ ]:
monthly_reports = {path[-13:-5]:load_and_process_df(path) for path in monthly_report_paths}

In [ ]:
for name, report in monthly_reports.items():
    generate_report(report, name)

## Concatenating data frames

In [ ]:
df_all = pd.concat((df for df in monthly_reports.values()))
df_all.head()

In [ ]:
top_selling_items(df_all)

In [ ]:
generate_report(df_all, 'all-data')

## Bonus: point-and-click data frames with [Qgrid](https://github.com/quantopian/qgrid)

In [ ]:
qgrid_widget = qgrid.show_grid(df,
                               show_toolbar=True,
                               grid_options={'forceFitColumns': False}
                              )
qgrid_widget

### Run this cell to get the updates into the original data frame

In [ ]:
qgrid_widget.get_changed_df()